# Clusters of Galaxies DEMO

# Table of Contents

* [Preliminary settings](#chapt0)
    * [Import necessary packages and set directories](#sec0.1)
    * [Create an instance of the class EuclidLikelihood](#sec0.2)
* [Validation of the HMF code](#chapt1)
* [Validation of the $\Delta\Sigma(R)$ codes](#chapt2)
    * [Centered profiles](#chapt2_sec1)
    * [Off-centered profiles](#chapt2_sec2)
* [Validation of the likelihood models](#chapt3)

## Import necessary packages and set directories <a class="anchor" id="sec0.1"></a>

In [ ]:
# General Python imports
import numpy as np
import os
import matplotlib.pyplot as plt
import sys

# Set the CLOE directory
likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)

# CAMB path
camb_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0,camb_path)

# Import external loglike from the Likelihood Package within cobaya_interface.py
from cloe.cobaya_interface import EuclidLikelihood
from cloe.user_interface.likelihood_ui import LikelihoodUI
from cloe.clusters_of_galaxies.CG import CG
from cloe.auxiliary.likelihood_yaml_handler \
import write_params_yaml_from_info_dict
from cloe.auxiliary.likelihood_yaml_handler \
import set_halofit_version

## Create an instance of the class EuclidLikelihood <a class="anchor" id="sec0.2"></a>

In [ ]:
# We are running the Euclid-Likelihood as an external likelihood class for Cobaya
# Cobaya needs a dictionary or yaml file as input to start running
# This dictionary below ('info') can be modified up to some point by the user to
# adapt it to the user's needs.
# The options that can be modified by the user are pointed with the acronym (UC).

info = {
    #'params': Cobaya's protected key of the input dictionary. 
    # Includes the parameters that the user would like to sample over:
    'params': {
        # (UC): each parameter below (which is a 'key' of another sub-dictionary) can contain a dictionary
        # with the key 'prior', 'latex'...
        # If the prior dictionary is not passed to a parameter, this parameter is fixed.
        # In this example, we are sampling the parameter ns
        # For more information see: https://cobaya.readthedocs.io/en/latest/example.html
        'ombh2': 0.022387993, #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': 0.120088016, #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': 67.32, #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': 0.0925, #optical depth
        'mnu': (0.3158-0.314378999)*93.04*pow(0.6732, 2), #  sum of the mass of neutrinos in eV
        #'mnu': 0.0, #  sum of the mass of neutrinos in eV 
        'nnu': 3.046, #N_eff of relativistic species 
        #'neutrino_cdm': 'cb',
        'As': 2.09175e-9, #Amplitude of the primordial scalar power spectrum
        #'As': 2.035e-9, #Amplitude of the primordial scalar power spectrum
        #'sigma8': 0.816,
        'ns': 0.9661, # primordial power spectrum tilt (sampled with an uniform prior)
        'w': -1, #Dark energy fluid model
        'wa': 0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        'omegam': None, #DERIVED parameter: Omega matter density
        'omegab': None, #DERIVED parameter: Omega barion density
        'omeganu': None, #DERIVED parameter: Omega neutrino density
        'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None,
    },
    #'theory': Cobaya's protected key of the input dictionary.
    # Cobaya needs to ask some minimum theoretical requirements to a Boltzman Solver
    # (UC): you can choose between CAMB or CLASS
    # In this DEMO, we use CAMB and specify some CAMB arguments
    # such as the number of massive neutrinos
    # and the dark energy model
    #
    # ATTENTION: If you have CAMB/CLASS already installed and 
    # you are not using the likelihood conda environment 
    # or option (2) in cell (3) (Cobaya modules), you can add an extra key called 'path' within the camb dictionary
    # to point to your already installed CAMB code
    'theory': {'camb': 
               {'stop_at_error': True, 
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf'}}},
    #'sampler': Cobaya's protected key of the input dictionary.
    # (UC): you can choose the sampler you want to use.
    # Check Cobaya's documentation to see the list of available samplers
    # In this DEMO, we use the 'evaluate' sampler to make a single computation of the posterior distributions
    # Note: if you want to run a simple MCMC sampling choose 'mcmc'
    'sampler': {'evaluate': None},  
    # 'packages_path': Cobaya's protected key of the input dictionary.
    # This is the variable you need to update
    # if you are running Cobaya with cobaya_modules (option (2) above).
    # If you are using the conda likelihood environment or option (1),
    # please, keep the line below commented
    #
    #'packages_path': modules_path,
    #
    #'output': Cobaya's protected key of the input dictionary.
    # Where are the results going to be stored, in case that the sampler produce output files? 
    # For example: chains...
    # (UC): modify the path below within 'output' to choose a name and a directory for those files
    'output': 'chains/my_euclid_experiment',
    #'likelihood': Cobaya's protected key of the input dictionary.
    # (UC): The user can select which data wants to use for the analysis.
    # Check Cobaya's documentation to see the list of the current available data experiments
    # In this DEMO, we load the Euclid-Likelihood as an external function, and name it 'Euclid'

    'likelihood': {'Euclid':
            {'external': EuclidLikelihood,
                'speed': 500,
                'k_max_extrap': 50.0,
                'k_min_extrap': 1e-05,
                'k_samp': 1000,
                'z_min': 0.0,
                'z_max': 4.0,
                'z_samp': 100,
                'solver': 'camb',
                'NL_flag_phot_matter': 0,
                'NL_flag_spectro': 0,
                'NL_flag_phot_baryon': 0,
                'NL_flag_phot_bias': 0,
                'IA_flag': 0,
                'IR_resum': 'DST',
                'Baryon_redshift_model': True,
                'GCsp_z_err': False,
                'bias_model': 1,
                'use_magnification_bias_spectro': 0,
                'use_Weyl': False,
                'magbias_model': 2,
                'use_gamma_MG': False,
                'f_out_z_dep': False,
                'plot_observables_selection': False,
                # Switch to allow for matrix transformations of theory and data vectors
                'data':
                {
                    'sample': 'ExternalBenchmark',
                    'CG':
                    {    
                        'statistics_clusters': 'cluster_counts',
                         'file_names_CC' :'data_CG_CC.dat',
                         'file_cov_names_CC' :'data_cov_CG_CC.dat',
                         'file_names_MoR' :'data_CG_MoR.dat',
                         'file_cov_names_MoR' :'data_cov_CG_MoR.dat',
                         'file_names_xi2' :'data_CG_xi2.npy',
                         'file_cov_names_xi2' :'data_cov_CG_xi2.npy'
                    },
                    'photo':
                    {   'redshifts': [0.2095,0.489,0.619,0.7335,0.8445,0.9595,1.087,1.2395,1.45,2.038],
                        'luminosity_ratio': 'luminosity_ratio.dat',
                        'IA_model': 'zNLA',
                        'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npy',
                        'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npy',
                        'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npy',
                        'cov_model': 'Gauss',
                        'cov_is_num': False,
                        'cov_nsim': 10000,
                        'ndens_GC': 'niTab-EP10-RB00.dat',
                        'ndens_WL': 'niTab-EP10-RB00.dat',
                        'root_GC': 'Cls_{:s}_PosPos.dat',
                        'root_WL': 'Cls_{:s}_ShearShear.dat',
                        'root_XC': 'Cls_{:s}_PosShear.dat',
                        'root_mixing_matrix': 'fs2_mms_10zbins_32ellbins.fits',
                        'Fourier': True,
                    },
                    'spectro':
                    {
                        'redshifts': ['1.', '1.2', '1.4', '1.65'],
                        'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                        'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                        'cov_is_num': False,
                        'cov_nsim': 3500,
                        'Fourier': True,
                    },                   
                },
                'observables_selection':
                {
                    'CG':
                    {
                        'CG': True,                        
                    },
                    'WL':
                    {
                        'WL': False,
                        'GCphot': False,
                        'GCspectro': False,
                    },
                    'GCphot':
                    {
                        'GCphot': False,
                        'GCspectro': False,
                    },
                    'GCspectro':
                    {
                        'GCspectro': False,
                    },                                      
                    'add_phot_RSD': False,
                    'matrix_transform_phot' : False, # 'BNT' or 'BNT-test'
                },
                'observables_specifications':
                {   
                    'CG':
                   {
                    'CG_probe': 'CC',
                    'CG_xi2_cov_selection': 'CG_nonanalytic_cov',
                    'neutrino_cdm': 'cb',
                    'external_richness_selection_function': 'non_CG_ESF',
                    'file_richness_selection_function': './../data/ExternalBenchmark/Clusters/int_Plob_ltr_z_Dlob.npy',
                    'effective_area': 10313,
                    'z_obs_edges_CC': np.array([0.2, 0.4, 0.6, 0.8, 1., 1.2, 1.4, 1.6]),
                    'Lambda_obs_edges_CC': np.array([20,30,45,60,500]),
                    'halo_profile': 'BMO',
                    'overdensity_type': 'vir',
                    'overdensity': 200.,
                    'A_l': 52.0,
                    'B_l': 0.9,
                    'C_l': 0.5,
                    'sig_A_l': 0.2,
                    'sig_B_l': -0.05,
                    'sig_C_l': 0.001,
                    'M_pivot': 3.e14 / 0.6732,
                    'z_pivot': 0.45,
                    'sig_lambda_norm': 0.9,
                    'sig_lambda_z': 0.1,
                    'sig_lambda_exponent': 0.4,
                    'sig_z_norm': 0.,
                    'sig_z_z': 0.025,
                    'sig_z_lambda': 5.e-6
                  },
                  'GCphot-GCspectro': None,
                  'WL-GCspectro': None
               },
          },
        },
    
    #'debug': Cobaya's protected key of the input dictionary.
    # (UC): how much information you want Cobaya to print? If debug: True, it prints every single detail
    # that is going on internally in Cobaya
    'debug': False,
    #'timing': Cobaya's protected key of the input dictionary.
    # (UC): if timing: True, Cobaya returns how much time it took it to make a computation of the posterior
    # and how much time take each of the modules to perform their tasks
    'timing': False,
    #'force': Cobaya's protected key of the input dictionary.
    # (UC): if 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    'force': True
    }

#write_params_yaml_from_info_dict(info)

In [ ]:
from cobaya.model import get_model # import the 'model' wrapper of Cobaya 
model = get_model(info)            # create an instance of the `model` wrapper called model

logposterior = model.logposterior({})
like = EuclidLikelihood() # Create an instance of the class EuclidLikelihood
like.passing_requirements(model, info, **model.provider.params) 
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)
like.initialize()

# Again for some reason
like.passing_requirements(model, info, **model.provider.params) 
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)

# Validation of the HMF code <a class="anchor" id="chapt1"></a>

In [ ]:
# Create an instance of the CG class
clusters = CG(like.cosmo.cosmo_dic)

# Declare the redshift array
zed = np.array([0.0000,0.2137248000388392,0.42515106601299735,0.673416111650323,0.9649067651739924,
                1.7090726844201227])

# Declare the true OmegaM(z), D(z), sigma8(z) arrays
OmegaM_true = np.array([3.158000e-01, 4.521322e-01, 5.719208e-01, 6.838367e-01, 7.778521e-01, 9.017372e-01])
Dz_true     = np.array([1., 8.925343e-01, 7.987282e-01, 7.052811e-01, 6.159577e-01, 4.594538e-01])
sigma8_true = np.array([0.8102, 0.7231, 0.6471, 0.5714, 0.4990, 0.3722])

# Set the path and file names for the validation tables
root_dir = './../data/ExternalBenchmark/Clusters/validation_tables/HMF_validation_tables'
filename = ['truth-table-Piccolo-00-z0.0.txt','truth-table-Piccolo-00-z0.2137248000388392.txt',
            'truth-table-Piccolo-00-z0.42515106601299735.txt','truth-table-Piccolo-00-z0.673416111650323.txt',
            'truth-table-Piccolo-00-z0.9649067651739924.txt','truth-table-Piccolo-00-z1.7090726844201227.txt']

# Set the Pk file name
Pk_dir      = root_dir+"/Pk-C0/"
Pk_fileroot = "Pk-PICCOLO-C0-z"
Pk_filename = ["0.0000","0.2137","0.4252","0.6734","0.9649","1.7091"]

# Declare h
h = like.cosmo.cosmo_dic['H0']/100.

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(12,10))
gs = fig.add_gridspec(3, 5, width_ratios=[2,0.5,1.5,0.5,2], height_ratios=[1,0.1,1])

fig.suptitle("Power spectrum", fontsize=18, y=1.05)

# D(z)
ax1 = fig.add_subplot(gs[0,0:2])
ax1.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax1.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax1.set_xlabel(r'$z$', color='black', fontsize=14)
ax1.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax1.set_title(r'$D(z)$', fontsize=14)

Dz_theo = [ like.cosmo.growth_factor(zed[i],0.05) for i in range(len(zed)) ]
ax1.plot(zed, (Dz_true-Dz_theo)/Dz_theo*100.)

# sigma8(z)
ax2 = fig.add_subplot(gs[0,3:5])
ax2.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax2.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax2.set_xlabel(r'$z$', color='black', fontsize=14)
ax2.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax2.set_title(r'$\sigma_8(z)$', fontsize=14)

sigma8_theo = [ like.cosmo.cosmo_dic['sigmaR_z_func'](zed[i],8./h,grid=False) for i in range(len(zed)) ]

ax2.plot(zed, (sigma8_true-sigma8_theo)/sigma8_theo*100.)

# Pk
ax4 = fig.add_subplot(gs[2,1:4])
ax4.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax4.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax4.set_xscale("log")
ax4.set_xlabel(r'$k$ [$h$ Mpc$^{-1}$]', color='black', fontsize=14)
ax4.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax4.set_title(r'$P\,(k)$ [Mpc$^3$ $h^{-3}$]', fontsize=14)

for i in range(len(Pk_filename)):
    k       = np.genfromtxt(os.path.join(Pk_dir,Pk_fileroot+Pk_filename[i]+".txt"), 
                            usecols=(0), dtype=None, skip_header = 2)
    Pk_true = np.genfromtxt(os.path.join(Pk_dir,Pk_fileroot+Pk_filename[i]+".txt"), 
                            usecols=(1), dtype=None, skip_header = 2)
    
    Pk_theo = clusters.Pk_def(zed[i],k*h,nu_cdm=0.) * h**3
    
    ax4.plot(k,(Pk_true-Pk_theo)/Pk_theo*100.0, label=r"$z=$"+str("{:.2f}".format(zed[i])))

ax4.legend(loc="lower left", prop={'size': 12})

plt.show()

In [ ]:
from scipy.special import gamma

# -----------------------------
# Define the HMF parameters

a1 = 0.7962
a2 = 0.1449
az = -0.0658
p1 = -0.5612
p2 = -0.4743
q1 = 0.3688
q2 = -0.2804
qz = 0.0251


# -----------------------------
# Compute and plot

fig = plt.figure(constrained_layout=True, figsize=(12,17))
gs = fig.add_gridspec(7, 5, width_ratios=[2,0.5,1.5,0.5,2], height_ratios=[1,0.1,1,0.1,1,0.1,1])

fig.suptitle("Intermediate equations for the computation of "+r"$f(\nu)$"+", and "+r"$f(\nu)$",
             fontsize=18, y=1.05)


# aR
ax0 = fig.add_subplot(gs[0,0:2])
ax0.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax0.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax0.set_xscale("log")
ax0.set_xlabel(r'$m$', color='black', fontsize=14)
ax0.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax0.set_title(r'$a_R$', fontsize=14)

for i in range(len(filename)):
    mass    = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(0), dtype=None, skip_header = 2)
    aR_true = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(5), dtype=None, skip_header = 2)
    aR_theo = a1 + a2 * (clusters.dlnsdlnR([zed[i]],mass/h)[0]+0.6125)**2.
    
    ax0.plot(mass,(aR_true-aR_theo)/aR_theo*100.0, label=r"$z=$"+str("{:.2f}".format(zed[i])))
    
ax0.legend(loc="lower left", prop={'size': 12})

# a
ax02 = fig.add_subplot(gs[0,3:5])
ax02.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax02.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax02.set_xscale("log")
ax02.set_xlabel(r'$m$', color='black', fontsize=14)
ax02.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax02.set_title(r'$a$', fontsize=14)

for i in range(len(filename)):
    mass    = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(0), dtype=None, skip_header = 2)
    a_true = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(6), dtype=None, skip_header = 2)
    
    aR_theo = a1 + a2 * (clusters.dlnsdlnR([zed[i]],mass/h)[0]+0.6125)**2.
    a_theo = aR_theo * clusters.Omm_z(zed[i], nu_cdm=0)**az
    
    ax02.plot(mass,(a_true-a_theo)/a_theo*100.0, label=r"$z=$"+str("{:.2f}".format(zed[i])))
    
ax02.legend(loc="lower left", prop={'size': 12})

# p
ax1 = fig.add_subplot(gs[2,0:2])
ax1.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax1.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax1.set_xscale("log")
ax1.set_xlabel(r'$m$', color='black', fontsize=14)
ax1.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax1.set_title(r'$p$', fontsize=14)

for i in range(len(filename)):
    mass    = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(0), dtype=None, skip_header = 2)
    p_true = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(7), dtype=None, skip_header = 2)
    p_theo = p1 + p2 * (clusters.dlnsdlnR([zed[i]],mass/h)[0]+0.5)
    
    ax1.plot(mass,(p_true-p_theo)/p_theo*100.0, label=r"$z=$"+str("{:.2f}".format(zed[i])))
    
ax1.legend(loc="lower right", prop={'size': 12})

# qR
ax2 = fig.add_subplot(gs[2,3:5])
ax2.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax2.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax2.set_xscale("log")
ax2.set_xlabel(r'$m$', color='black', fontsize=14)
ax2.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax2.set_title(r'$q_R$', fontsize=14)

for i in range(len(filename)):
    mass    = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(0), dtype=None, skip_header = 2)
    qR_true = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(8), dtype=None, skip_header = 2)
    qR_theo = q1 + q2 * (clusters.dlnsdlnR([zed[i]],mass/h)[0]+0.5)
    
    ax2.plot(mass,(qR_true-qR_theo)/qR_theo*100.0, label=r"$z=$"+str("{:.2f}".format(zed[i])))
    
ax2.legend(loc="upper right", prop={'size': 12})

# q
ax3 = fig.add_subplot(gs[4,0:2])
ax3.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax3.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax3.set_xscale("log")
ax3.set_xlabel(r'$m$', color='black', fontsize=14)
ax3.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax3.set_title(r'$q$', fontsize=14)

for i in range(len(filename)):
    mass    = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(0), dtype=None, skip_header = 2)
    q_true = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(9), dtype=None, skip_header = 2)
    q_theo = (q1 + q2 * (clusters.dlnsdlnR([zed[i]],mass/h)[0]+0.5)) * clusters.Omm_z(zed[i], nu_cdm=0)**qz
    
    ax3.plot(mass,(q_true-q_theo)/q_theo*100.0, label=r"$z=$"+str("{:.2f}".format(zed[i])))
    
ax3.legend(loc="upper right", prop={'size': 12})

# A
ax4 = fig.add_subplot(gs[4,3:5])#(gs[4,1:4])
ax4.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax4.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax4.set_xscale("log")
ax4.set_xlabel(r'$m$', color='black', fontsize=14)
ax4.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax4.set_title(r'$A(p,q)$', fontsize=14)

for i in range(len(filename)):
    mass    = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(0), dtype=None, skip_header = 2)
    A_true = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(4), dtype=None, skip_header = 2)
    
    p  = p1 + p2 * (clusters.dlnsdlnR([zed[i]],mass/h)[0]+0.5)
    qR = q1 + q2 * (clusters.dlnsdlnR([zed[i]],mass/h)[0]+0.5)
    q  = qR * clusters.Omm_z(zed[i], nu_cdm=0)**qz
    A_theo = 1. / (2.**(-0.5-p+q/2.)/np.sqrt(np.pi) * (2.**p*gamma(q/2.) + gamma(-p+q/2.)))
    
    ax4.plot(mass,(A_true-A_theo)/A_theo*100.0, label=r"$z=$"+str("{:.2f}".format(zed[i])))
    
ax4.legend(loc="upper right", prop={'size': 12})

# f(nu)
ax5 = fig.add_subplot(gs[6,1:4])
ax5.xaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax5.yaxis.set_tick_params(direction='in', which='both', labelsize=12)
ax5.set_xscale("log")
ax5.set_xlabel(r'$m$', color='black', fontsize=14)
ax5.set_ylabel('(truth - CLOE) / CLOE x 100', color='black', fontsize=14)
ax5.set_title(r'$\nu f(\nu)$', fontsize=14)

for i in range(len(filename)):
    mass    = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(0), dtype=None, skip_header = 2)
    nufnu_true = np.genfromtxt(os.path.join(root_dir,filename[i]), usecols=(2), dtype=None, skip_header = 2)
    nufnu_theo = clusters.f_sigma_nu(np.array([zed[i]]),mass/h)[0]
    
    ax5.plot(mass,(nufnu_true-nufnu_theo)/nufnu_theo*100.0, label=r"$z=$"+str("{:.2f}".format(zed[i])))
    
ax5.legend(loc="lower left", prop={'size': 12})

plt.show()

# Validation of the likelihood models <a class="anchor" id="chapt3"></a>

In [ ]:
input_dir = "./../data/ExternalBenchmark/Clusters/truth_tables/numbercounts_truth_tables/"

# Bins for cluster counts
l_ob_mean = np.array([23.8714427 , 35.64512875, 51.13983791, 79.60670823])
l_ob_edges = np.array([20,30,45,60,500])
z_ob_edges = np.array([0.2, 0.4, 0.6, 0.8, 1., 1.2, 1.4, 1.6])

# Bins for weak-lensing
theta_ob_edges = np.load(input_dir+"theta_bins_4gammatob_data_lam_lg20_02_z_16.npy")

# Bins for clustering
r_ob_edges_xi = np.geomspace(20,130,31) * (like.cosmo.cosmo_dic['H0']/100.)
l_ob_edges_xi = np.array([20,30,500])
z_ob_edges_xi = np.arange(0.2, 1.81, 0.4)

# Compute the model with CLOE
clusters = CG(like.cosmo.cosmo_dic)

model_theo = clusters.N_zbin_Lbin_Rbin()

In [ ]:
# Read measures and expectation values
measure = np.load(input_dir+"NC_data_lam_lg20_02_z_16.npy")
model_tr = np.genfromtxt(input_dir+"NC_CG_CC_model.dat")

# Set the plot
fig, ax = plt.subplots(6, 3, figsize=(16,18), sharey="row",
                       gridspec_kw={"hspace":0, "wspace":0, "height_ratios":[1,0.5,1,0.5,1,0.5]})
for i in range(len(ax)):
    for j in range(len(ax[i])):
        ax[i][j].xaxis.set_tick_params(direction='in', which='both', labelsize=14)
        ax[i][j].yaxis.set_tick_params(direction='in', which='both', labelsize=14)
        ax[i][j].set_xscale("log")
        if i % 2 == 0:
            ax[i][j].set_yscale("log")
        if j == 0:
            if i % 2 == 0:
                ax[i][j].set_ylabel(r"$N\,(\Delta\lambda^{\rm ob},\Delta z^{\rm ob})$", fontsize=16)
            else:
                ax[i][j].set_ylabel(r"% diff.", fontsize=16)            

fig.delaxes(ax[4][1])
fig.delaxes(ax[5][1])
fig.delaxes(ax[4][2])
fig.delaxes(ax[5][2])

ax[5][0].set_xlabel(r"$\lambda^{\rm ob}$", fontsize=18)
ax[3][1].set_xlabel(r"$\lambda^{\rm ob}$", fontsize=18)
ax[3][2].set_xlabel(r"$\lambda^{\rm ob}$", fontsize=18)

# Plot
def plot_validation(ax1, ax2, z_idx):
    
    if z_idx == 0:
        labelspacing = 0.5
        label_measure = "Measure"
        label_model_tr = "Reference model"
        label_model_theo = "CLOE model"
    else:
        labelspacing = 0.
        label_measure = None
        label_model_tr = None
        label_model_theo = None
    
    # Measure
    ax1.errorbar(l_ob_mean, measure[z_idx], np.sqrt(measure[z_idx]), 
                 color="k", capsize=5, fmt="o", label=label_measure)
    
    # Reference model
    ax1.plot(l_ob_mean, model_tr[z_idx], color="k", linestyle="solid", label=label_model_tr)
    
    # CLOE model
    ax1.plot(l_ob_mean, model_theo[0][z_idx], color="r", linestyle="dashed", label=label_model_theo)
    
    ax1.legend(loc="upper right", fontsize=13, title_fontsize=13, labelspacing=labelspacing, 
               title=r"$z\in["+str(z_ob_edges[z_idx])+","+str(z_ob_edges[z_idx+1])+"]$")
    
    # Plot % differences
    ax2.plot(l_ob_mean, (measure[z_idx]-model_theo[0][z_idx])/model_theo[0][z_idx]*100, 
             color="b", linestyle="dashed", label="CLOE vs Measure")
    ax2.plot(l_ob_mean, (model_tr[z_idx]-model_theo[0][z_idx])/model_theo[0][z_idx]*100, 
             color="g", linestyle="solid", label="CLOE vs Reference model")
    
    if z_idx == 0:
        ax2.legend(loc="lower center", fontsize=12)

z_idx = 0
for i in range(len(ax[0])):
    plot_validation(ax[0][i], ax[1][i], z_idx)
    z_idx += 1
for i in range(len(ax[2])):
    plot_validation(ax[2][i], ax[3][i], z_idx)
    z_idx += 1
plot_validation(ax[4][0], ax[5][0], z_idx)

plt.show()